In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')


In [6]:
def get_data(
        csvs_root='../../csvs/',
        rca_n_name='RCA_n.csv',
        data_root='../../../../data/trade/BACI/'):
    # RCA BASE TABLE
    rca_n = pd.read_csv(
        csvs_root  + rca_n_name,
        dtype={'product': object, 'flow':int})

    # COMMODITY CODE
    commodity_code = pd.read_csv(
        csvs_root + 'commodity_code.csv',
        encoding='latin-1',
        dtype={'Code': object, 'isLeaf':int, 'Level':int})
    # BACI uses SITC rev 3(S3)
    commodity_code = \
        commodity_code[commodity_code['Classification']=='S3']
    # drop last column
    commodity_code = commodity_code.iloc[:, :-1]

    # COUNTRY NAMES
    country_df = pd.read_csv(data_root + 'countries.csv')
    countries = country_df.Name.values
    countries = np.append(countries, ["United States of America", "China, Hong Kong SAR", "China, Taiwan Province of", "Türkiye", "Iran (Islamic Republic of)", "Czechia", "Switzerland, Liechtenstein", "China, Macao SAR", "Korea, Dem. People's Rep. of", "Venezuela (Bolivarian Rep. of)", "Côte d'Ivoire", "Congo, Dem. Rep. of the", "Lao People's Dem. Rep.", "Bolivia (Plurinational State of)", "North Macedonia", "Curaçao", "State of Palestine", "Cabo Verde", "Eswatini", "British Virgin Islands", "Micronesia (Federated States of)", "Wallis and Futuna Islands", "Holy See",])
    return rca_n, commodity_code, countries

# 2017 data

In [20]:
# load data 
csvs_root = '../../csvs/'
data_root = '../../../../data/trade/BACI/'
rca_n_xaa_2017, commodity_code, countries = \
    get_data(csvs_root, 'rca_n_xaa_2017.csv', data_root)
rca_n_xab_2017, commodity_code, countries = \
    get_data(csvs_root, 'rca_n_xab_2017.csv', data_root)

# merge xaa and xab
rca_n = pd.concat([rca_n_xaa_2017, rca_n_xab_2017])
rca_n.drop_duplicates(inplace=True)

# merge
commodity_code.rename(columns={'Code': 'product'}, inplace=True)
rca_n = rca_n.merge(commodity_code, on='product', how='left')

## exports only

In [21]:
# only exports 
rca_n = rca_n[rca_n.flow==2].copy()

# only countries
rca_n_countries = rca_n[rca_n.economy_label.isin(countries)].copy()

# drop columns that aren't included in commodity codes
rca_n_countries.dropna(subset=['Classification'], inplace=True)

### some data missing between level1 and 2,3but we will work with 3

In [23]:
rca_n_countries.groupby(['Level'])['sum_kusd'].sum().reset_index()

,Level,sum_kusd
0,0.0,4.855826e+11
1,1.0,4.853860e+11
2,2.0,4.724147e+11
3,3.0,4.724147e+11


In [24]:
level3_filter = rca_n_countries.Level==3
rca_n_countries = rca_n_countries[level3_filter].copy()

## calc RCA

In [25]:
# product/sum(product) for each country

sum_per_n = rca_n_countries.groupby(['economy_label'])\
    .agg({'sum_kusd': 'sum'})\
        .reset_index()[['economy_label', 'sum_kusd']]
sum_per_n = sum_per_n.rename({'sum_kusd': 'sum_per_n'}, axis=1)[['economy_label', 'sum_per_n']]

rca_n_countries = pd.merge(rca_n_countries, sum_per_n, on='economy_label', how='left')
rca_n_countries['n_ratio'] = rca_n_countries.apply(lambda x: x['sum_kusd']/x['sum_per_n'], axis=1)


# product/sum(product) across countries

sum_per_p = rca_n_countries.groupby(['product'])\
    .agg({'sum_kusd':'sum'})\
        .reset_index()[['product', 'sum_kusd']]
sum_per_p = sum_per_p.rename({'sum_kusd': 'sum_per_p'}, axis=1)[['product', 'sum_per_p']] # renaming for merging

rca_n_countries = pd.merge(rca_n_countries, sum_per_p, on='product', how='left')

# total of exports
sum_p = rca_n_countries['sum_kusd'].sum()
rca_n_countries['p_ratio'] = rca_n_countries.apply(lambda x: x['sum_per_p']/sum_p, axis=1)

rca_n_countries.sort_values(by=['economy_label','product'], inplace=True)
rca_n_countries.reset_index(drop=True, inplace=True)
rca_n_countries.to_csv('tmp.csv', index=False)


### diff of sum(products) vs TOTAL

In [28]:
economy_name = 'United States of America'
economy_name = 'China'
rca_n_countries_economy_filter = (rca_n_countries.economy_label==economy_name).values
rca_n_economy_filter = (rca_n.economy_label==economy_name).values
calculated_sum_per_n = rca_n_countries[rca_n_countries_economy_filter]['sum_per_n'].unique()
assert len(calculated_sum_per_n) == 1
actual_sum_per_n = rca_n[(rca_n['Level']==0).values & rca_n_economy_filter]['sum_kusd'].values[0]
difference = calculated_sum_per_n[0] - actual_sum_per_n
print(economy_name)
print("(calculated - actual)/actual: ", round(difference/actual_sum_per_n, 3)*100
, "%;\n", " "*29,round(difference/1e6), "million dollars")

China
(calculated - actual)/actual:  -0.2 %;
                               -144 million dollars


In [27]:
china_filter = (rca_n_countries['economy_label'] == 'China').values
usa_filter = (rca_n_countries['economy_label'] == 'United States of America').values
ukraine_filter = (rca_n_countries['economy_label'] == 'Ukraine').values
russia_filter = (rca_n_countries['economy_label'] == 'Russian Federation').values
rca_filter = (rca_n_countries['n_ratio'] > rca_n_countries['p_ratio']).values

china_rca = rca_n_countries[china_filter & rca_filter]
usa_rca = rca_n_countries[usa_filter & rca_filter]

china_rca_top5 = china_rca.sort_values(by=['n_ratio'], ascending=False).head(5)
usa_rca_top5 = usa_rca.sort_values(by=['n_ratio'], ascending=False).head(5)

china_rca_top5_names = china_rca_top5['product'].values
usa_rca_top5_names = usa_rca_top5['product'].values

# get products to make product network from 
top5_union = np.concatenate((china_rca_top5_names, usa_rca_top5_names))
top5_union = np.unique(top5_union)

In [49]:
top5_union

array(['222', '334', '728', '752', '759', '764', '778', '784', '821',
       '874'], dtype=object)

In [47]:
# compare with 2016
top5_union_2016 = ['222', '334', '752', '764', '778', '784', '821', '851', '874']
# what dissapeared from 2016 to 2017
dissapeared = set(top5_union_2016) - set(top5_union) 
dissapeared = list(dissapeared)
new = set(top5_union) - set(top5_union_2016)
new = list(new)
for product in dissapeared:
    print('dissapeared : ', end='')
    print(rca_n[rca_n['product'] == product]['product_label'].values[0])
for product in new:
    print('new         : ', end='')
    print(rca_n[rca_n['product'] == product]['product_label'].values[0])

dissapeared : Footwear
new         : Other machinery for particular industries, n.e.s.
new         : Parts, accessories for machines of groups 751, 752
